In [1]:
import pandas as pd 
import os
import spacy
import string
import re
import numpy as np
from sklearn.metrics import f1_score
from spacy.symbols import ORTH
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence 
from sklearn.model_selection import train_test_split



pd.options.mode.chained_assignment = None

In [2]:
# memory footprint support libraries/code
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize

# import psutil
# import humanize
# import os
# import GPUtil as GPU

# GPUs = GPU.getGPUs()
# gpu = GPUs[0]
# def printm():
#     process = psutil.Process(os.getpid())
#     print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), 
#           " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
#     print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(
#         gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))


In [3]:
torch.cuda.empty_cache()
#printm()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
#!ls '/content/drive/My Drive/app'

In [5]:
mimic_data = pd.read_csv("/content/drive/My Drive/app/text_binary.csv")
mimic_data.head()

,study_id,subject_id,text,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,58522792,16567081,"b"" FINAL REPOR...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,58213163,16567081,b' FINAL REPOR...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,59835582,16043746,b' FINAL REPOR...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,51487790,16456872,b' FINAL REPOR...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,59750073,16824069,b' FINAL REPOR...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [6]:
mimic_data = mimic_data.loc[mimic_data['No Finding'] == 0]
mimic_data.head()

,study_id,subject_id,text,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,58522792,16567081,"b"" FINAL REPOR...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,58213163,16567081,b' FINAL REPOR...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,50629293,16059088,"b"" FINAL REPOR...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
6,50113713,16059088,b' WET READ: ___ ___ ___ 7:27 PM\n Left PICC ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
7,57620089,16059088,b' FINAL REPOR...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [7]:
binary_data = mimic_data[['text','Pneumonia']]
binary_data.text = binary_data.text.str.lower()
binary_data.head()

,text,Pneumonia
0,"b"" final repor...",1.0
1,b' final repor...,0.0
5,"b"" final repor...",0.0
6,b' wet read: ___ ___ ___ 7:27 pm\n left picc ...,0.0
7,b' final repor...,0.0


In [8]:
binary_data.groupby('Pneumonia').count()

,text
Pneumonia,
0.0,135816
1.0,16556


In [9]:
negative = binary_data.loc[binary_data['Pneumonia']==0].sample(40000)
print(len(negative))
positive = binary_data.loc[binary_data['Pneumonia']==1].sample(n=len(negative),replace=True)

40000


In [10]:
binary_data = positive.append(negative,ignore_index=True).reset_index(drop=True)
#df1.append(df2, ignore_index=True)

In [11]:
len(binary_data)

80000

In [12]:
binary_data.text[0]

"b'                                 final report\\n examination:  chest (portable ap)\\n \\n indication:  ___ year old woman with pna and intubated  // ett placement     \\n ett placement\\n \\n impression: \\n \\n compared to chest radiographs ___ through ___.\\n \\n severe left upper lobe pneumonia, improved on ___ compared to ___, subsequently stable.  left lower lobe consolidation has worsened, and\\n bronchopneumonia or focal edema in the right lower lung is still moderately\\n severe.  pleural effusions are presumed but not large.  heart is top-normal\\n size.  no pneumothorax.\\n \\n et tube in standard placement.  right jugular line ends in the low svc. \\n nasogastric drainage tube ends in the upper portion of a stomach distended\\n with semi-solid material.\\n'"

In [13]:
y = binary_data['Pneumonia']
X = binary_data.drop(columns=['Pneumonia'])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)

In [14]:
X_train.head()

,text
0,b' final repor...
1,b' final repor...
2,b' final repor...
3,b' final repor...
4,b' final repor...


In [15]:
# remove all '\\n' from the text
re_newlines = re.compile('\\\\n')
def sub_newlines(x): return re_newlines.sub('',x)

# remove all special characters from the text, keep only alphanumeric and spaces
re_letters = re.compile('[^A-Za-z0-9 ]')
def sub_letters(x): return re_letters.sub('', x)

# remove excessive spacing otherwise you end up with " " substrings
re_spaces = re.compile('\s+')
def sub_spaces(x): return re_spaces.sub(' ', x)
                
# tokenize all words.
my_tok = spacy.load('en')
def spacy_tok(x): 
    return [tok.text for tok in my_tok.tokenizer(sub_spaces
                                                 (sub_letters
                                                 (sub_newlines(x))))]

In [16]:
sub_spaces(sub_letters(sub_newlines(binary_data.text[0])))

'b final report examination chest portable ap indication year old woman with pna and intubated ett placement ett placement impression compared to chest radiographs through severe left upper lobe pneumonia improved on compared to subsequently stable left lower lobe consolidation has worsened and bronchopneumonia or focal edema in the right lower lung is still moderately severe pleural effusions are presumed but not large heart is topnormal size no pneumothorax et tube in standard placement right jugular line ends in the low svc nasogastric drainage tube ends in the upper portion of a stomach distended with semisolid material'

In [17]:
#spacy_tok(binary_data.text[0])[1:]

In [18]:
binary_train = binary_data.copy()
binary_train.head()

,text,Pneumonia
0,b' final repor...,1.0
1,b' final repor...,1.0
2,b' final repor...,1.0
3,"b"" final adden...",1.0
4,b' final repor...,1.0


In [19]:
def get_counts(text):
    counts = Counter()
    for word in text:
        counts.update(spacy_tok(word)[1:])
    return counts

In [20]:
word_count = get_counts(binary_train.text)

In [21]:
len(word_count.keys())

20033

In [22]:
for word in list(word_count):
    if word_count[word] < 3:
        del word_count[word]

In [23]:
len(word_count.keys())

11220

In [24]:
vocab2index = {"<PAD>":0, "UNK":1} # init with padding and unknown
words = ["<PAD>", "UNK"]
for word in word_count:
    vocab2index[word] = len(words)
    words.append(word)

In [25]:
len(words)

11222

In [26]:
def encode_sentence(text):
    enc = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in text.split()])
    return enc

In [27]:
encode_sentence(binary_train.text[0])

array([ 1,  2,  1,  1,  5,  1,  1,  1,  1,  1,  9, 10, 11, 12, 13, 14, 15,
        1, 16, 17,  1, 16,  1,  1,  1,  1,  1, 19, 20,  5, 21,  1, 22,  1,
        1, 23, 24, 25, 26,  1, 28, 29,  1, 19, 20,  1, 30,  1, 24, 32, 26,
       33, 34,  1,  1, 36, 37, 38, 39, 40, 41, 42, 32, 43, 44, 45,  1,  1,
       47, 48, 49, 50, 51, 52,  1, 54, 44,  1,  1, 57,  1,  1, 59, 60, 40,
       61,  1, 42, 62, 63, 64, 40, 41, 65,  1,  1, 67, 68, 60, 64, 40, 41,
       25, 69, 70, 71, 72,  1, 12,  1,  1])

## Dataset and data loaders

In [28]:
class Binary_Mimic(Dataset):
    def __init__(self, X, y, vocab):
        self.x = [encode_sentence(x) for x in X.text]
        self.y = y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        return x, y

In [29]:
b_train = Binary_Mimic(X_train, y_train, vocab2index)
b_valid = Binary_Mimic(X_val, y_val, vocab2index)

In [30]:
def collate_fn(data):
    """Creates mini-batch tensors from the list of tuples (sentences, labels).
    
    Need custom collate_fn because merging sequences (including padding) is not 
    supported in default. Sequences are padded to the maximum length of mini-batch 
    sequences (dynamic padding).
    
    Args:
        data: list of tuple (sentence, label). 
            - list of word indices of variable length
            - label, 0 or 1
    Returns:
        packed_batch: (PackedSequence), see torch.nn.utils.rnn.pack_padded_sequence
        sencences: torch tensor of shape (batch_size, max_len).
        labels: torch tensor of shape (batch_size, 1).
        lengths: list; valid length for each padded sentence. 
    """
    # Sort a data list by sentences length (descending order).
    data.sort(key=lambda x: len(x[0]), reverse=True)
    sentences, labels = zip(*data)
    
    # stack labels
    labels = torch.Tensor(labels)
    
    # Merge sentences
    lengths = [len(s) for s in sentences]
   
    sents = torch.zeros(len(sentences), max(lengths)).long()
    for i, s in enumerate(sentences):
        end = lengths[i]
        sents[i, :end] = torch.Tensor(s[:end])        
    
    return sents, lengths, labels

In [31]:
# data = [b_train[0], b_train[1], b_train[2]]
# data

In [32]:
# collate_fn(data)

In [33]:
# train_loader = DataLoader(b_train, batch_size=2, shuffle=True, collate_fn=collate_fn)
# sents, lengths, labels = next(iter(train_loader))
# sents

In [34]:
# sents.shape, labels.shape

In [35]:
# lengths

## Training

In [36]:
def train_epocs(model, optimizer, train_dl, valid_dl, epochs=10):
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, s, y in train_dl:
            x = x.cuda().long()
            y = y.cuda().float()
            y_pred = model(x, s)
            optimizer.zero_grad()
            loss = F.binary_cross_entropy_with_logits(y_pred, y.unsqueeze(1))
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc = val_metrics(model, valid_dl)
        print("Epoch #%.f: train loss %.3f val loss %.3f and val accuracy %.3f" % 
              (i+1,sum_loss/total, val_loss, val_acc))
  

In [37]:
def val_metrics(model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    with torch.no_grad():
      for x, s, y in valid_dl:
          x = x.cuda().long()
          y = y.cuda().float().unsqueeze(1)
          y_hat = model(x, s)
          loss = F.binary_cross_entropy_with_logits(y_hat, y)
          y_pred = y_hat > 0
          correct += (y_pred.float() == y).float().sum()
          total += y.shape[0]
          sum_loss += loss.item()*y.shape[0]
    return sum_loss/total, correct/total

## Basic GRU Model

In [38]:
class GRUModel(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super(GRUModel,self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x, lengths):
        #print(x.shape)
        x = self.embeddings(x)
        x = self.dropout(x)
        pack = pack_padded_sequence(x, lengths, batch_first=True)
        out_pack, ht = self.gru(pack)
        return self.linear(ht[-1])

In [39]:
torch.cuda.empty_cache()
#printm()

In [40]:
batch_size = 25000
train_dl = DataLoader(b_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_dl = DataLoader(b_valid, batch_size=batch_size, collate_fn=collate_fn)

In [41]:
vocab_size = len(words)
print(vocab_size)
model = GRUModel(vocab_size, 50, 50).cuda()

parameters = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.Adam(parameters, lr=0.01)

11222


In [42]:
train_epocs(model, optimizer, train_dl, valid_dl, epochs=15)

Epoch #1: train loss 0.712 val loss 0.688 and val accuracy 0.502
Epoch #2: train loss 0.690 val loss 0.671 and val accuracy 0.570
Epoch #3: train loss 0.664 val loss 0.651 and val accuracy 0.623
Epoch #4: train loss 0.645 val loss 0.622 and val accuracy 0.649
Epoch #5: train loss 0.612 val loss 0.591 and val accuracy 0.699
Epoch #6: train loss 0.576 val loss 0.558 and val accuracy 0.737
Epoch #7: train loss 0.541 val loss 0.516 and val accuracy 0.759
Epoch #8: train loss 0.514 val loss 0.479 and val accuracy 0.786
Epoch #9: train loss 0.478 val loss 0.462 and val accuracy 0.801
Epoch #10: train loss 0.452 val loss 0.433 and val accuracy 0.811
Epoch #11: train loss 0.431 val loss 0.415 and val accuracy 0.821
Epoch #12: train loss 0.406 val loss 0.405 and val accuracy 0.830
Epoch #13: train loss 0.390 val loss 0.380 and val accuracy 0.838
Epoch #14: train loss 0.375 val loss 0.367 and val accuracy 0.843
Epoch #15: train loss 0.359 val loss 0.361 and val accuracy 0.850


## Use Glove to create pre-trained embedding 

In [43]:
def loadGloveModel(gloveFile='/content/drive/My Drive/app/glove.6B.50d.txt'):
    """ Loads word vectors into a dictionary."""
    f = open(gloveFile,'r')
    word_vecs = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        word_vecs[word] = np.array([float(val) for val in splitLine[1:]])
    return word_vecs

In [44]:
word_vecs = loadGloveModel()

In [45]:
word_count = get_counts(binary_train.text)

In [46]:
print(len(word_vecs.keys()), len(word_count.keys()))

400000 20033


In [47]:
def delete_rare_words(word_vecs, word_count, min_df=4):
    """ Deletes rare words from word_count
    
    Deletes words from word_count if they are not in word_vecs
    and don't have at least min_df occurrencies in word_count.
    """
    words_delete = []
    for word in word_count:
        if word_count[word] < min_df and word not in word_vecs:
            words_delete.append(word)
    for word in words_delete: word_count.pop(word)
    return word_count

In [48]:
word_count = delete_rare_words(word_vecs, word_count, min_df=3)
print(len(word_count.keys()))

14280


In [49]:
def create_embedding_matrix(word_vecs, word_count, min_df=4, emb_size=50):
    """Creates embedding matrix from word vectors. """
    word_count = delete_rare_words(word_vecs, word_count, min_df)
    V = len(word_count.keys()) + 2
    vocab2index = {}
    W = np.zeros((V, emb_size), dtype="float32")
    vocab = ["", "UNK"]
    # adding a vector for padding
    W[0] = np.zeros(emb_size, dtype='float32')
    # adding a vector for rare words 
    W[1] = np.random.uniform(-0.25, 0.25, emb_size)
    vocab2index["UNK"] = 1
    i = 2
    for word in word_count:
        if word in word_vecs:
            W[i] = word_vecs[word]
            vocab2index[word] = i
            vocab.append(word)
            i += 1
        else:
            W[i] = np.random.uniform(-0.25,0.25, emb_size)
            vocab2index[word] = i
            vocab.append(word)
            i += 1   
    return W, np.array(vocab), vocab2index

In [50]:
pretrained_weight, vocab, vocab2index = create_embedding_matrix(word_vecs, word_count, min_df=3)

In [51]:
len(pretrained_weight)

14282

In [52]:
emb_size = 50
V = len(pretrained_weight)
emb = nn.Embedding(V, emb_size)
emb.weight.data.copy_(torch.from_numpy(pretrained_weight))

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1331, -0.1580,  0.0825,  ...,  0.0766,  0.1868, -0.0453],
        [-0.4124,  0.6493, -0.5585,  ...,  0.2621,  0.1045, -0.4430],
        ...,
        [ 1.5811, -0.1405, -0.5253,  ..., -0.0395, -0.6693,  0.3181],
        [-0.5752, -0.0844,  0.5022,  ...,  0.6287,  0.0503,  0.8334],
        [ 0.4725, -0.3981,  0.1524,  ..., -1.1998, -0.4230,  0.6049]])

# GRU Model with Pretrained embedding layer:

In [53]:
class GRUModel_pre(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim, glove_weights=None) :
        super(GRUModel_pre,self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        if glove_weights is not None:
            self.embeddings.weight.data.copy_(torch.from_numpy(glove_weights))
            self.embeddings.weight.requires_grad = False ## freeze embeddings
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x, lengths):
        x = self.embeddings(x)
        x = self.dropout(x)
        pack = pack_padded_sequence(x, lengths, batch_first=True)
        out_pack, ht = self.gru(pack)
        return self.linear(ht[-1])

In [54]:
torch.cuda.empty_cache()
#printm()

In [55]:
vocab_size = len(pretrained_weight)
model = GRUModel_pre(vocab_size, 50, 50, glove_weights=pretrained_weight).cuda()

parameters = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.Adam(parameters, lr=0.01)

In [56]:
train_epocs(model, optimizer,train_dl, valid_dl, epochs=5)

Epoch #1: train loss 0.689 val loss 0.680 and val accuracy 0.573
Epoch #2: train loss 0.677 val loss 0.670 and val accuracy 0.588
Epoch #3: train loss 0.666 val loss 0.649 and val accuracy 0.623
Epoch #4: train loss 0.656 val loss 0.637 and val accuracy 0.637
Epoch #5: train loss 0.653 val loss 0.651 and val accuracy 0.619


In [57]:
model.embeddings.weight.requires_grad = True

In [58]:
parameters = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.Adam(parameters, lr=0.01)
train_epocs(model, optimizer, train_dl, valid_dl, epochs=15)

Epoch #1: train loss 0.665 val loss 0.638 and val accuracy 0.639
Epoch #2: train loss 0.647 val loss 0.630 and val accuracy 0.645
Epoch #3: train loss 0.628 val loss 0.596 and val accuracy 0.688
Epoch #4: train loss 0.597 val loss 0.559 and val accuracy 0.721
Epoch #5: train loss 0.551 val loss 0.485 and val accuracy 0.780
Epoch #6: train loss 0.508 val loss 0.456 and val accuracy 0.800
Epoch #7: train loss 0.476 val loss 0.469 and val accuracy 0.796
Epoch #8: train loss 0.449 val loss 0.417 and val accuracy 0.824
Epoch #9: train loss 0.423 val loss 0.396 and val accuracy 0.833
Epoch #10: train loss 0.408 val loss 0.400 and val accuracy 0.832
Epoch #11: train loss 0.390 val loss 0.383 and val accuracy 0.840
Epoch #12: train loss 0.377 val loss 0.361 and val accuracy 0.849
Epoch #13: train loss 0.366 val loss 0.356 and val accuracy 0.852
Epoch #14: train loss 0.357 val loss 0.355 and val accuracy 0.854
Epoch #15: train loss 0.345 val loss 0.341 and val accuracy 0.858
